In [1]:
import re
from optparse import OptionParser
from pyflann import *
import cv2
import numpy as np
import pandas as pd
import glob
import os

meta_data_header = ['FrameNumber', 'Time', 'Stage_x', 'Stage_y', 'Centroid_x', 'Centroid_y',
                    'Midpoint_x', 'Midpoint_y', 'Head_x', 'Head_y', 'Tail_x', 'Tail_y', 'MouthHook_x', 'MouthHook_y',
                    'LeftMHhook_x', 'LeftMHhook_y', 'RightMHhook_x', 'RightMHhook_y',
                    'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y', 'RightDorsalOrgan_x', 'RightDorsalOrgan_y',
                    'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y', 'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y',
                    'RightBolwigOrgan_x', 'RightBolwigOrgan_y', 'V9_x', 'V9_y', 'V10_x', 'V10_y', 'V11_x', 'V11_y',
                    'V12_x', 'V12_y', 'V13_x', 'V13_y', 'V14_x', 'V14_y', 'V15_x', 'V15_y', 'V16_x', 'V16_y',
                    'MouthHook_votes', 'LeftMHhook_votes', 'RightMHhook_votes', 'LeftDorsalOrgan_votes',
                    'RightDorsalOrgan_votes', 'CenterBolwigOrgan_votes', 'LeftBolwigOrgan_votes', 'RightBolwigOrgan_votes',
                    'V9_votes', 'V10_votes', 'V11_votes', 'V12_votes', 'V13_votes', 'V14_votes', 'V15_votes',
                    'V16_votes', 'Num_Key_points']


In [2]:
test_dir = '../expts/dataCollectedOn_20180417_withTestOn_20180619/dataCollectedOn_2018041_grp_2/20180529_150807/Rawdata_20180417_082627_20180525_112343_A_133410/'
video_path = '../expts/dataCollectedOn_20180417_withTestOn_20180619/videos/'

test_string = "Rawdata_20180417_082627"
video_file = os.path.join(video_path, test_string+".avi")
test_string_2 = str.split(test_string, '_')[2]

crop_size = 512
spot_size = 25.0
font = cv2.FONT_HERSHEY_SIMPLEX

for fs in os.listdir(test_dir):
    if 'Metadata' in fs:
        meta_data = pd.read_csv(os.path.join(test_dir, fs), sep=',', header=0, names=meta_data_header)

if (meta_data.empty is False):
    start_frame = 149
    cap = cv2.VideoCapture(video_file)

    numberFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    meta_data.loc[:, 'FrameNumber'] = meta_data.loc[:, 'FrameNumber'] + start_frame
    meta_data.set_index('FrameNumber', inplace=True)

    outputFrameSize = np.multiply(np.ones((1920, 1920, 3), dtype=np.uint8), 255)
    height, width, layers = outputFrameSize.shape
    
    outputFrameSize_crop = np.multiply(np.ones((crop_size, crop_size, 3), dtype=np.uint8), 255)
    height_crop, width_crop, layers_crop = outputFrameSize_crop.shape

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    outputVideoFile = os.path.join(test_dir, '_'.join(str.split(test_string, '_')[0:3]) + '.avi')
    outputVideoFile_crop = os.path.join(test_dir, '_'.join(str.split(test_string, '_')[0:3]) + '_crop.avi')    

    video = cv2.VideoWriter(outputVideoFile, 0, 5, (width, height))
    video_crop = cv2.VideoWriter(outputVideoFile_crop, 0, 5, (width_crop, height_crop))

    if cap.isOpened():
        for row_index, row in meta_data.iterrows():
            cap.set(1, row_index)
            
            ret, originalFrame = cap.read()

            frame = originalFrame.copy()
            frame_overlay = originalFrame.copy()

            cropCenter_X = row['Head_x']
            cropCenter_Y = row['Head_y']
            
            cv2.circle(frame_overlay, (int(cropCenter_X), int(cropCenter_Y)),
                       radius=10, color=(0, 0, 0), thickness=-1)
            
            ## MouthHook
            cv2.rectangle(frame_overlay, (int(row['MouthHook_x']) - int(spot_size/2),
                                  int(row['MouthHook_y']) - int(spot_size/2)),
                          (int(row['MouthHook_x']) + int(spot_size/2),
                           int(row['MouthHook_y']) + int(spot_size/2)),
                          color=(255, 0, 0), thickness=-1)
            
            ## Left Dorsal Organ
            cv2.rectangle(frame_overlay, (int(row['LeftDorsalOrgan_x']) - int(spot_size/2),
                                  int(row['LeftDorsalOrgan_y']) - int(spot_size/2)),
                          (int(row['LeftDorsalOrgan_x']) + int(spot_size/2),
                           int(row['LeftDorsalOrgan_y']) + int(spot_size/2)),
                          color=(125, 125, 255), thickness=-1)
            
            ## Right Dorsal Organ
            cv2.rectangle(frame_overlay, (int(row['RightDorsalOrgan_x']) - int(spot_size/2),
                                  int(row['RightDorsalOrgan_y']) - int(spot_size/2)),
                          (int(row['RightDorsalOrgan_x']) + int(spot_size/2),
                           int(row['RightDorsalOrgan_y']) + int(spot_size/2)),
                          color=(0, 255, 0), thickness=-1)            
            
            cv2.putText(frame, '%d'%(row_index), (1720, 200), font, 2, (0, 0, 0), 4, cv2.LINE_AA)
            
            crop_x = int(max(0, cropCenter_X-int(crop_size/2)))
            crop_y = int(max(0, cropCenter_Y-int(crop_size/2)))

            frame_crop = frame[crop_y:crop_y+crop_size, crop_x:crop_x+crop_size]
            frame_crop_overlay = frame_overlay[crop_y:crop_y+crop_size, crop_x:crop_x+crop_size]
            
            cv2.putText(frame_crop, '%d'%(row_index), (360, 40), font, 0.65, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame_crop, '%s - %d' % ('MH', row['MouthHook_votes']), (360, 60), font, 0.65, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame_crop, '%s - %d' % ('LDO', row['LeftDorsalOrgan_votes']), (360, 80), font, 0.65, (125, 125, 255), 2, cv2.LINE_AA)
            cv2.putText(frame_crop, '%s - %d' % ('RDO', row['RightDorsalOrgan_votes']), (360, 100), font, 0.65, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame_crop, 'Spot - %d um' % (spot_size*2.75), (360, 120), font, 0.65, (0, 0, 0), 2, cv2.LINE_AA)
            
            alpha = 0.3
            cv2.addWeighted(frame_crop_overlay, alpha, frame_crop, 1 - alpha, 0, frame_crop)
            video_crop.write(frame_crop)

    cv2.destroyAllWindows()
    video.release()